<a href="https://colab.research.google.com/github/white9812/notimportant_ai/blob/main/0729%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%B1%84%EC%9A%B0%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#합쳐진 데이터셋 불러오기
concat_csv=os.listdir("/content/drive/MyDrive/filled_data/filled_data")

In [ ]:
concat_csv

['강원도.csv',
 '경기도.csv',
 '경상남도.csv',
 '대구시.csv',
 '경상북도.csv',
 '광주시.csv',
 '서울시.csv',
 '부산시.csv',
 '대전시.csv',
 '울산시.csv',
 '세종시.csv',
 '전라남도.csv',
 '전라북도.csv',
 '인천시.csv',
 '충청남도.csv',
 '제주도.csv',
 '충청북도.csv']

In [ ]:
del concat_csv[:9]
len(concat_csv)

10

In [ ]:

concat_Df=[]
for concat_csv_path in concat_csv:


    path="/content/drive/MyDrive/filled_data/filled_data/"
    c_df=pd.read_csv(path+f"{concat_csv_path}") #station의 데이터 이름에 데이터 프레임 저장하기
    concat_Df.append(c_df)



In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

from tqdm import tqdm


In [ ]:
for corr_df,name_csv in zip(concat_Df,concat_csv):
  corr_data=corr_df[[ 'Solar_Power(MWh)', '일사', '일조', '습도', '지면온도', '시정', '기온','중하층운량', '풍속', '전운량', '강수량', '해면기압', '풍향', '적설', '현지기압', '증기압', '최저운고','이슬점온도']].corr()
  df_imputed_corr=round(corr_data.abs(),2)
  correlations_solar_power =df_imputed_corr.loc[:, "Solar_Power(MWh)"]
  high_correlations = correlations_solar_power[correlations_solar_power >= 0.2]
  corr_list=high_correlations.index.to_list()
  sampledata=corr_df[corr_list].copy()
    # Custom IterativeImputer to show progress
  class IterativeImputerWithProgress(IterativeImputer):
      def __init__(self, estimator=None, missing_values=np.nan, sample_posterior=False, max_iter=30,
                  tol=1e-3, n_nearest_features=None, initial_strategy="mean", imputation_order='ascending',
                  skip_complete=False, min_value=None, max_value=None, verbose=0, random_state=None, add_indicator=False):
          super().__init__(estimator=estimator, missing_values=missing_values, sample_posterior=sample_posterior,
                          max_iter=max_iter, tol=tol, n_nearest_features=n_nearest_features, initial_strategy=initial_strategy,
                          imputation_order=imputation_order, skip_complete=skip_complete, min_value=min_value,
                          max_value=max_value, verbose=verbose, random_state=random_state, add_indicator=add_indicator)
          self._tqdm = None

      def fit_transform(self, X, y=None):
          self._tqdm = tqdm(total=self.max_iter, desc='Imputation Progress')
          result = super().fit_transform(X, y)
          self._tqdm.close()
          return result

      def _impute_one_feature(self, X_filled, mask_missing_values, feat_idx, neighbor_feat_idx, estimator=None, fit_mode=True):
          imputed_values = super()._impute_one_feature(X_filled, mask_missing_values, feat_idx, neighbor_feat_idx, estimator, fit_mode)
          if fit_mode:
              self._tqdm.update(1)
          return imputed_values

  # RandomForestRegressor 설정 조정
  reduced_rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0)

  # 배치 크기 설정
  batch_size = len(sampledata)

  # 배치별로 MissForest 알고리즘 적용
  imputed_batches = []
  n_batches = int(np.ceil(len(sampledata) / batch_size))

  for i in range(n_batches):
      batch = sampledata.iloc[i * batch_size:(i + 1) * batch_size]
      imputer = IterativeImputerWithProgress(estimator=reduced_rf, max_iter=30, random_state=0, add_indicator=False)
      imputed_batch = imputer.fit_transform(batch)
      # 반환된 imputed_batch의 형태 확인
      print(f"Batch {i+1} shape: {imputed_batch.shape}")

      # 데이터프레임으로 변환
      imputed_batch_df = pd.DataFrame(imputed_batch, columns=sampledata.columns[:imputed_batch.shape[1]])
      imputed_batches.append(imputed_batch_df)



  # 배치 결합
  df_imputed = pd.concat(imputed_batches, axis=0).reset_index(drop=True)

  #데이터 내보내기
  df_imputed.to_csv(f"0729_pm_filed_{name_csv}",index=False)






Imputation Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Imputation Progress:   3%|▎         | 1/30 [00:06<03:15,  6.73s/it]

Imputation Progress:   7%|▋         | 2/30 [00:13<03:16,  7.02s/it]

Imputation Progress:  10%|█         | 3/30 [00:21<03:17,  7.33s/it]

Imputation Progress:  13%|█▎        | 4/30 [00:28<03:04,  7.11s/it]

Imputation Progress:  17%|█▋        | 5/30 [00:35<02:57,  7.10s/it]

Imputation Progress:  20%|██        | 6/30 [00:41<02:45,  6.88s/it]

Imputation Progress:  23%|██▎       | 7/30 [00:49<02:42,  7.08s/it]

Imputation Progress:  27%|██▋       | 8/30 [00:54<02:24,  6.59s/it]

Imputation Progress:  30%|███       | 9/30 [01:01<02:20,  6.68s/it]

Imputation Progress:  33%|███▎      | 10/30 [01:08<02:16,  6.80s/it]

Imputation Progress:  37%|███▋      | 11/30 [01:16<02:15,  7.13s/it]

Imputation Progress:  40%|████      | 12/30 [01:23<02:05,  6.99s/it]

Imputation Progress:  43%|████▎     | 13/30 [01:30<02:00,  7.06s/it]

Imputation Progress:  47%|████▋     

Batch 1 shape: (53999, 8)




Imputation Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Imputation Progress:   3%|▎         | 1/30 [00:07<03:39,  7.55s/it]

Imputation Progress:   7%|▋         | 2/30 [00:14<03:20,  7.16s/it]

Imputation Progress:  10%|█         | 3/30 [00:21<03:12,  7.11s/it]

Imputation Progress:  13%|█▎        | 4/30 [00:28<02:58,  6.88s/it]

Imputation Progress:  17%|█▋        | 5/30 [00:35<02:56,  7.08s/it]

Imputation Progress:  20%|██        | 6/30 [00:42<02:47,  6.99s/it]

Imputation Progress:  23%|██▎       | 7/30 [00:49<02:41,  7.00s/it]

Imputation Progress:  27%|██▋       | 8/30 [00:56<02:32,  6.93s/it]

Imputation Progress:  30%|███       | 9/30 [01:02<02:23,  6.84s/it]

Imputation Progress:  33%|███▎      | 10/30 [01:09<02:15,  6.76s/it]

Imputation Progress:  37%|███▋      | 11/30 [01:17<02:14,  7.08s/it]

Imputation Progress:  40%|████      | 12/30 [01:23<02:02,  6.82s/it]

Imputation Progress:  43%|████▎     | 13/30 [01:30<01:59,  7.02s/it]

Imputation Progress:  47%|████▋     

Batch 1 shape: (53999, 9)


Imputation Progress:  37%|███▋      | 11/30 [01:23<02:19,  7.32s/it]